<a href="https://colab.research.google.com/github/elyasncr/Curso_Machine_learning/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dados Uttilizados do Kaggle
* https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud


In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [78]:
#Dataset
dados = pd.read_csv('creditcard.csv')

In [79]:
dados.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [80]:
#Quantos dados retornam como nulo nas colunas
dados.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [81]:
# Número de transações
n_transacoes = dados['Class'].count()
n_transacoes
print(f'Número de transações: {n_transacoes}')

Número de transações: 284807


In [82]:
# Quantidade de transações fruadulentas
n_fraudes = dados['Class'].sum()
n_fraudes
print(f'Número de fraudes: {n_fraudes}')

Número de fraudes: 492


In [83]:
# Quantidade de transações normais
n_normais = n_transacoes - n_fraudes
n_normais
print(f'Número de transações normais: {n_normais}')

Número de transações normais: 284315


In [84]:
# percentual de transações fraudulentas
percen_fraudes = (n_fraudes / n_transacoes)
percen_fraudes
print("Porcentagem de fraudes: ", "%.2f" %(percen_fraudes*100), "%")


Porcentagem de fraudes:  0.17 %


In [85]:
# Percentual de transações normais
percen_normais = (n_normais / n_transacoes) * 100
percen_normais
print("Porcentagem de transações normais: ", "%.2f" %(percen_normais), "%")

Porcentagem de transações normais:  99.83 %


#### Começando a treinar os dados

In [86]:
# Criando a divisão para teste e treino através de um validador
def validador(X, y):
    validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
    for treino_id, teste_id in validador.split(X,y):
        X_treino = X[treino_id]
        y_treino = y[treino_id]
        X_teste = X[teste_id]
        y_teste = y[teste_id]
    return X_treino, X_teste, y_treino, y_teste


In [95]:
%%time
def exec_classificador(classificador, X_treino, y_treino, X_teste):
    # Passando os dados de treinamento no classificador
    arvore = classificador.fit(X_treino, y_treino)
    # Identificando nas variáveis de teste as possíveis fraudes
    y_pred = arvore.predict(X_teste)
    return y_pred

In [89]:
def salvar_arvore(classificador, nome):
    plt.figure(figsize=(200,100))
    tree.plot_tree(classificador, filled=True, fontsize=14)
    plt.savefig(nome)
    plt.close()

In [121]:
%%time
# Validando os valores da nossa árvore de decisão
def validar_arvore(y_teste, y_pred):
    print(confusion_matrix(y_teste, y_pred))
    print("Acuracia: ", (accuracy_score(y_teste, y_pred)))
    print("Precisão: ", (precision_score(y_teste, y_pred)))
    print("Recall: ", (recall_score(y_teste, y_pred)))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


In [96]:
# X todos os dados da tabela, exceto a coluna Class
X = dados.drop('Class', axis=1).values

# y coluna alvo, então foi separada para observação
y = dados['Class'].values

X_treino, X_teste, y_treino, y_teste = validador(X, y)

In [97]:
# executando o classificador
classificador_arvore = tree.DecisionTreeClassifier()

In [129]:
%%time
y_pred_decision = exec_classificador(classificador_arvore, X_treino, y_treino, X_teste)

CPU times: user 26.1 s, sys: 15.4 ms, total: 26.1 s
Wall time: 29.8 s


In [100]:
# exibindo a árvore de decisão
salvar_arvore(classificador_arvore, 'arvore_decisao.png')

In [130]:
%%time
# Validação da árcore de decisão
validar_arvore(y_teste, y_pred_decision)

[[28430     2]
 [   13    36]]
Acuracia:  0.9994733330992591
Precisão:  0.9473684210526315
Recall:  0.7346938775510204
CPU times: user 46.7 ms, sys: 906 µs, total: 47.6 ms
Wall time: 60.8 ms


A matriz de confusão acima mostra o seguinte aspecto:
* ##### na diagonal principal temos:
   * 28421 foram os acertos sem fraude
   * 36 foram os acertos com fraude
* #### na diagonal secundária temos:
   * 2 foram os erros sem fraude
   * 13 foram os erros com fraude

Além disso, temos também:
* Acurácia de: 99%
* Precisão de: 94%
* Recall de: 73%


In [131]:
# Caracteristicas da árvore de decisão
print(classificador_arvore)
print(classificador_arvore.get_depth())
print(classificador_arvore.get_n_leaves())

DecisionTreeClassifier(max_depth=10, random_state=0)
10
63


In [132]:
%%time
# executando o classificador com 10 ramificações
classificador_arvore = tree.DecisionTreeClassifier(max_depth=10, random_state=0)

y_pred_decision = exec_classificador(classificador_arvore, X_treino, y_treino, X_teste)

CPU times: user 28.6 s, sys: 22.8 ms, total: 28.6 s
Wall time: 37 s


In [133]:
%%time
# Validando a árvore com max_depth igual a 10
validar_arvore(y_teste, y_pred_decision)

[[28430     2]
 [   13    36]]
Acuracia:  0.9994733330992591
Precisão:  0.9473684210526315
Recall:  0.7346938775510204
CPU times: user 40.3 ms, sys: 921 µs, total: 41.3 ms
Wall time: 42 ms


In [134]:
%%time
# executando o classificador com 10 ramificações e 10 instâncias em cada nó
classificador_arvore = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)

y_pred_decision = exec_classificador(classificador_arvore, X_treino, y_treino, X_teste)

CPU times: user 27.4 s, sys: 23.2 ms, total: 27.4 s
Wall time: 31 s


In [135]:
%%time
# Validando a árvore com max_depth igual a 10 e 10 instâncias em cada nó
validar_arvore(y_teste, y_pred_decision)

[[28426     6]
 [   12    37]]
Acuracia:  0.9993679997191109
Precisão:  0.8604651162790697
Recall:  0.7551020408163265
CPU times: user 50.8 ms, sys: 0 ns, total: 50.8 ms
Wall time: 61.6 ms


Ao compararmos o validador incial com este, vemos que a precisão decaiu em relação ao primeiro. Neste caso, o min_samples_leaf = 5 não nos ajudou a melhorar a precisão. Entretanto, o recall melhorou, mas não é a nossa variável foco.

In [136]:
%%time
# executando o classificador com 5 ramificações
classificador_arvore = tree.DecisionTreeClassifier(max_depth=5, random_state=0)
y_pred_decision = exec_classificador(classificador_arvore, X_treino, y_treino, X_teste)

CPU times: user 14.9 s, sys: 21.9 ms, total: 15 s
Wall time: 20.4 s


In [137]:
validar_arvore(y_teste, y_pred_decision)

[[28429     3]
 [   14    35]]
Acuracia:  0.999403110845827
Precisão:  0.9210526315789473
Recall:  0.7142857142857143


Comparando com o caso acima, removemos o min_samples_leaf e diminuimos a quantidade de ramificações de 10 para 5, contudo a precisão e o recall caíram em comparação com o inicial. Sendo assim, o max_depth = 5 não é uma boa escolha para o nosso modelo de estudo.


In [139]:
%%time
# Usando o Random Forest
classificador_random = RandomForestClassifier(n_estimators=100, random_state=0)
y_pred_random = exec_classificador(classificador_random, X_treino, y_treino, X_teste)


CPU times: user 8min 27s, sys: 488 ms, total: 8min 28s
Wall time: 9min 2s


In [140]:
salvar_arvore(classificador_random.estimators_[0], 'arvore_random1.png')
salvar_arvore(classificador_random.estimators_[1], 'arvore_random2.png')

In [141]:
#validando o Random Forest
validar_arvore(y_teste, y_pred_random)

[[28430     2]
 [   12    37]]
Acuracia:  0.9995084442259752
Precisão:  0.9487179487179487
Recall:  0.7551020408163265


In [142]:
%%time
# Usando o Random Forest
classificador_random = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
y_pred_random = exec_classificador(classificador_random, X_treino, y_treino, X_teste)


CPU times: user 2min 27s, sys: 121 ms, total: 2min 27s
Wall time: 2min 32s


In [143]:
#validando o Random Forest
validar_arvore(y_teste, y_pred_random)

[[28431     1]
 [   12    37]]
Acuracia:  0.9995435553526912
Precisão:  0.9736842105263158
Recall:  0.7551020408163265


In [145]:
%%time
# Classificando com Adaboost
calssificador_ada = AdaBoostClassifier(random_state=0)
y_pred_ada = exec_classificador(calssificador_ada, X_treino, y_treino, X_teste)

CPU times: user 2min 24s, sys: 164 ms, total: 2min 24s
Wall time: 2min 42s


In [146]:
# Gerando a imagem da árvore feita no AdaBoost
salvar_arvore(calssificador_ada.estimators_[0], 'arvore_ada1.png')
salvar_arvore(calssificador_ada.estimators_[1], 'arvore_ada2.png')

In [147]:
# Validando o AdaBoost
validar_arvore(y_teste, y_pred_ada)

[[28428     4]
 [   17    32]]
Acuracia:  0.9992626663389628
Precisão:  0.8888888888888888
Recall:  0.6530612244897959


In [148]:
%%time
# Classificando com Adaboost com 100 ramificações
calssificador_ada = AdaBoostClassifier(random_state=0, n_estimators=100)
y_pred_ada = exec_classificador(calssificador_ada, X_treino, y_treino, X_teste)

CPU times: user 4min 49s, sys: 309 ms, total: 4min 49s
Wall time: 5min 5s


In [149]:
%%time
# Validando o AdaBoost
validar_arvore(y_teste, y_pred_ada)

[[28426     6]
 [   11    38]]
Acuracia:  0.999403110845827
Precisão:  0.8636363636363636
Recall:  0.7755102040816326
CPU times: user 38.9 ms, sys: 1.01 ms, total: 39.9 ms
Wall time: 39 ms
